# MongoDB NoteBook

> Cette partie montre les démarches pour tester six différentes opérations dans une mongo base. Les sources de données sont smartGrid et éolienne.  

## Installation et configuration
Télécharger et désarchiver le package de `mongoDB` pour Centos avec ligne commande : 
```
wget http://downloads.mongodb.org/linux/mongodb-linux-x86_64-rhel70-4.2.3.tgz
tar -xvfz mongodb-linux-x86_64-rhel70-4.2.3.tgz
```
Si mongodb est installé avec l'archive, il faut créer le fichier de configuration manuellement dans le répertoire `bin` et le nommer `mongodb.conf`.
Voici [format de fichier](https://github.com/mongodb/mongo/blob/master/rpm/mongod.conf).

Configurer le fichier mongodb.conf pour éviter le problème de privilège :
1. `path`: /work/weiy/local/mongodb-linux-x86_64-rhel70-4.2.3/bin/logs/mongodb.log
2. `dbPath` : /work/weiy/local/mongodb-linux-x86_64-rhel70-4.2.3/bin/data/db
3. `pidFilePath`: /work/weiy/local/var/run/mongodb/mongod.pid
4. `port`: 28018 
5. `authorization`: disabled

### Installer mongodb client

Pour communiquer avec MongoDB en utilisant son protocole TCP par défaut sur le port 27017, il a besoin d'un client MongoDB.
Ouvrir un terminal et taper la ligne commande suivante:
```bash
> sudo apt install mongodb-clients
```

Vérifier l'installation : 

```bash
> mongo --version
```

### Installer le client python

Installer le package `pymongo` (le client python pour mongoDB) avec ligne commande :
```
python -m pip install pymongo
```
>Pour vérifier les données dans mongoDB, [`mongo-compass`](https://www.mongodb.com/products/compass) est une GUI officielle qui permet des interactions avec mongoDB. 

## Connexion de mongodb
Ouvrir le terminal et aller dans le répertoire `bin` de mongodb.
Lancer mongoDB selon le fichier de configuration avec ligne commande : 
```
./mongod --config mongodb.conf
```
Pour lancer mongo shell, utiliser ligne commande :
```
./mongo --port 28018
```
Pour lancer le service de mongodb sans fichier de configuration, utiliser ligne commande :
```
./mongod --port 28018 --dbpath /work/weiy/local/mongodb-linux-x86_64-rhel70-4.2.3/bin/data/db
```
Code exemple en python : 
```python
from pymongo import MongoClient,errors
client = MongoClient('localhost', 28018, serverSelectionTimeoutMS = 2000)
```
## Tester la performance

**Remarque** : Avant l’exécution du script, il faut d'abord lancer les services nécessaires :
 - kafka zookeeper : 
```bash
> bin/zookeeper-server-start.sh config/zookeeper.properties
```
 - kafka service
```bash
> bin/kafka-server-start.sh config/server.properties
```
 - mongoDB
```bash
> ./mongod --port 28018 --dbpath /work/weiy/local/mongodb-linux-x86_64-rhel70-4.2.3/bin/data/db
```
 - jaeger
```bash
> ./jeager-all-in-one
```
 - mongo-compass (optionnel)
```bash
> ./mongo-compass
```


### Exécution des tests
Tous les fonctions de tests sont écrits dans le fichier `../TimeSeriesTools/mongodb_test.py`. Pour exécuter la fonction à tester, il faut utiliser les lignes commandes dans un terminal ou exécuter les cellules avec les paramètres d'entrée dans le fichier `../MongoDBNotebook/mongoDB_inference_test.ipynb` avec jupyter-notebook. 

Liste des paramètres d'entrée : 
 - `--query`: le string de la requête
   - valeur par défaut : '{"timestamp":{"$regex":"10:"}}'
 - `--value`: la requête des modifications des données sélectionnées
    - valeur par défaut : '{"$set":{"timestamp":"00/00/0000 00:00"}}
 - `--topic`: indiquer le nom du topic désiré
    - valeur par défaut : "eolienne_DT"
 - `--collection`: indiquer le nom de la collection désirée
    - valeur par défaut : "eolienne_DT"
 - `--tracer`: indiquer le nom du tracer à initialiser
    - valeur par défaut : "mongodb_test_eolienne_1_jour"
 - `--domain`: indiquer l'address du domaine pour connecter à la base de mongodb
    - valeur par défaut : "localhost"
 - `--port`: indiquer la porte pour connecter à la base de mongodb
    - valeur par défaut : "27017"
 - `--function`: indiquer le nom de la fonction à tester, il n'a pas de valeur par défaut donc cet paramètre est **obligatoire**. 

**Remarque** : 
- Sauf que le numéro de la porte est int, toutes les types des paramètres d'entrée sont string.
- Les paramètres suivantes sont optionnels pour toutes les functions : 
    - <--tracer>
    - <--collection>
    - <--domain>
    - <--port>

Dans le script python il y a 6 fonctions de test à choisir :

- `test_insert_bulk` : Mesurer le temps pour insérer toutes les données collectées dans la base de donnée à la fois.
  - paramètres optionnels: <--topic>
- `test_insert_one` : Mesurer le temps pour insérer toutes les données collectées dans la base de donnée ligne par ligne.
  - paramètres optionnels: <--topic>
- `find_some_data` : Mesurer le temps pour trouver les données correspondantes à la requête dans la base de donnée.
  - paramètres optionnels: <--query>
- `find_all_data` : Mesurer le temps pour lire toutes les données dans la base de donnée.

- `update_some_data` : Mesurer le temps pour modifier les données correspondantes à la requête dans la base de donnée.
  - paramètres optionnels: <--query>,<--value>
- `update_all_data`: Mesurer le temps pour modifier toutes les données dans la base de donnée.
  - paramètres optionnels: <--query>,<--value>
  
### Code exemple 
Pour récupérer toutes les données venant d'un topic de kafka, et mesurer le temps d'insertion ligne par ligne dans la base de mongodb, il faut préciser au minimun un paramètre : le nom de la function à tester. Les autres cinq paramètres sont optionnels dans cette function pour indiquer le source des données (c'est à dire lire les données dans quel topic), la destination des données (insérer les données dans quelle collection dans la base de données), le nom de tracer pour visualiser les informations détailles de cette opération, le domain et la porte pour connecter à la base de mongodb (ça dépend le fichier de configuration).
Dans le terminal : 

```bash
> python mongodb_test.py --function test_insert_one --topic "eolienne_DT" --collection "eolienne_DT" --domain "localhost" --port 27017 --tracer "test_insert_lines"
```
Dans le notebook :

```python
%run ../TimeSeriesTools/mongodb_test.py --function test_insert_one --topic "eolienne_DT" --collection "eolienne_DT" --domain "localhost" --port 27017 --tracer "test_insert_lines"
```

Pendant l'exécution du script, une ligne de données incorrecte est insérée pour tester la validation de données.
Une message s'affiche dans la console qui indique l'index de ligne et les champs des données invalides: 

```bash
line:  51288 , position:  0 , require  Heure
line:  51288 , position:  1 , require  Temps écoulé
line:  51288 , position:  12 , require  MWD Wind Speed
```
Après l'exécution du script, un tracer qui contient deux sous-span (un pour l'étape de la collection des données venant du topic, l'autre pour l'étape de l'insertion des donnnées) est signalé. Les informations et les barres intéractives sont ensuite disponibles sur wen UI de jeager (http://localhost:16686 par défaut).

```bash
Reporting span bb054b92a0316e58:16b5632098eb1b2:44a4e06b3541cefc:1 test_insert_lines.collect_data
Reporting span bb054b92a0316e58:d0fa76c1b3f2abed:44a4e06b3541cefc:1 test_insert_lines.insert_one
51289  documents inserted
Reporting span bb054b92a0316e58:44a4e06b3541cefc:0:1 test_insert_lines.test_line_insertion
```

### Surveillance des tâches
Un span unique de tracer principal est initialisé avant le démarre de chaque fonction, ce span contient le temps d’exécution de la fonction, les tags et les logs.
Naviguer dans l'URL `http://localhost:16686/` (porte 16686 par defaut) pour acceder le Web UI de jaeger.
Choisir le nom de tracer à gauche et cliquer query pour trouver les informations dans le conteneur de ce tracer.

![test](/uploads/ea9513e752cfe4ba667e4475a813bdb9/test.png)





## Information des dépendances 
| Dépendances | Version |
| ------ | -----------: |
| mongoDB | rhel70-4.2.3 |
| pymongo | 3.0.1 |
| mongo-compass | 1.20.5 |

